In [5]:
import re
import copy
import numpy as np
from pprint import pprint

In [6]:
class Blueprint():
    
    def __init__(self, blueprint):
        p = r'Blueprint (\d+): Each ore robot costs (\d+) ore\. ' \
            r'Each clay robot costs (\d+) ore\. ' \
            r'Each obsidian robot costs (\d+) ore and (\d+) clay\. ' \
            r'Each geode robot costs (\d+) ore and (\d+) obsidian\.'
        
        match = re.match(p, blueprint)
        
        self.num = int(match.group(1))
        
        self.costs={'ore': [int(match.group(2)), 0, 0],
                   'clay': [int(match.group(3)), 0, 0],
                   'obsidian': [int(match.group(4)), int(match.group(5)), 0],
                   'geode': [int(match.group(6)), 0, int(match.group(7))]}
        
        self.maxcost={'ore': max([int(match.group(3)), int(match.group(4)), int(match.group(6))]),
                    'clay': int(match.group(5)),
                    'obsidian': int(match.group(7)), 'geode': 100}
        

    def can_build(self, t, have, ore, clay, obsidian):
                    
        if have >= self.maxcost[t]:
            return False
        o, c, ob = self.costs[t]
        return ore >= o and clay >= c and obsidian >= ob
        

class State():
    
    def __init__(self, blueprint):
        
        self.ore = 0
        self.clay = 0
        self.obsidian = 0
        self.geodes = 0
        self.building = ''
        self.time = 1
        
        self.bp = blueprint
        
        self.orebots = 1
        self.claybots = 0
        self.obsidianbots = 0
        self.geodebots = 0
        
    def get_hash(self, time):
        
        return f'{self.building}_{self.ore}_{self.clay}_{self.obsidian}_{self.geodes}' \
               f'_{self.orebots}_{self.claybots}_{self.obsidianbots}_{self.geodebots}'
        
    def step(self):
        
        if self.building != '':
            # Build a robot
            o, c, ob = self.bp.costs[self.building]
            self.ore -= o
            self.clay -= c
            self.obsidian -= ob
            
        # Collect ore
        
        self.ore += self.orebots
        self.clay += self.claybots
        self.obsidian += self.obsidianbots
        self.geodes += self.geodebots
                
        # Finish building the robot
        if self.building == 'ore':
            self.orebots += 1
        if self.building == 'clay':
            self.claybots += 1
        if self.building == 'obsidian':
            self.obsidianbots += 1
        if self.building == 'geode':
            self.geodebots += 1
            
        # Now check what we can do
        r = ['']
        if self.bp.can_build('geode', self.geodebots, self.ore, self.clay, self.obsidian):
            return ['geode']
            # r.append('geode')
        if self.bp.can_build('obsidian', self.obsidianbots, self.ore, self.clay, self.obsidian):
            r.append('obsidian')
        if self.bp.can_build('clay', self.claybots, self.ore, self.clay, self.obsidian):
            r.append('clay')
        if self.bp.can_build('ore', self.orebots, self.ore, self.clay, self.obsidian):
            r.append('ore')

        return r

In [7]:
def loop(tnow, tmax, states, state_set):
    
    maxgeodes = 0

    nextstates = []
    
    best = 0
    
    for state in states:
        nextbuild = state.step()
        for nb in nextbuild:
            if state.geodes >= best:
                best = state.geodes
            else:
                continue
            newstate = copy.deepcopy(state)
            newstate.building = nb
            h = newstate.get_hash(tnow)
            if not h in state_set:
                nextstates.append(newstate)
                state_set.add(h)

    if tnow == (tmax-2):
        print('Almost out of time')
        # out of time
        for s in nextstates:
            final_geodes = s.geodes + s.geodebots*2
            if s.building == 'geode':
                final_geodes += 1
            # if s.ore + s.orebots > blueprint.costs['geode'][0] and \
            #     s.obsidian + s.obsidianbots > blueprint.costs['geode'][2]:
            #     final_geodes += 1
            if final_geodes >= maxgeodes:
                maxgeodes = final_geodes
        return maxgeodes

    return loop(tnow+1, tmax, nextstates, state_set)

In [8]:
with open('input19a.txt', 'r') as inf:
    data = [line.strip() for line in inf]
    
n_geodes = []
quality = 0

for i, bp in enumerate(data):
    blueprint = Blueprint(bp)
    print('Blueprint', blueprint.num)
    print(blueprint.costs)
    states = [State(blueprint)]
    state_set = set()
    state_set.add(states[0].get_hash(0))

    geodes = loop(1, 24, states, state_set)
    print('Max geodes', geodes)
    n_geodes.append(geodes)
    quality += blueprint.num * geodes
    
print(n_geodes)
    
print('Quality', quality)

Blueprint 1
{'ore': [4, 0, 0], 'clay': [2, 0, 0], 'obsidian': [3, 14, 0], 'geode': [2, 0, 7]}
Almost out of time
Max geodes 9
Blueprint 2
{'ore': [2, 0, 0], 'clay': [3, 0, 0], 'obsidian': [3, 8, 0], 'geode': [3, 0, 12]}
Almost out of time
Max geodes 12
[9, 12]
Quality 33


In [12]:
with open('input19.txt', 'r') as inf:
    data = [line.strip() for line in inf]
    
n_geodes = []
quality = 0

for i in range(3):
    blueprint = Blueprint(data[i])
    print('Blueprint', blueprint.num)
    print(blueprint.costs)
    states = [State(blueprint)]
    state_set = set()
    state_set.add(states[0].get_hash(0))

    geodes = loop(1, 32, states, state_set)
    print('Max geodes', geodes)
    n_geodes.append(geodes)
    
print(n_geodes)
    
print(n_geodes[0] * n_geodes[1] * n_geodes[2])


Blueprint 1
{'ore': [4, 0, 0], 'clay': [4, 0, 0], 'obsidian': [4, 17, 0], 'geode': [4, 0, 16]}
Almost out of time
Max geodes 8
Blueprint 2
{'ore': [4, 0, 0], 'clay': [4, 0, 0], 'obsidian': [4, 20, 0], 'geode': [2, 0, 8]}
Almost out of time
Max geodes 16
Blueprint 3
{'ore': [2, 0, 0], 'clay': [3, 0, 0], 'obsidian': [3, 13, 0], 'geode': [3, 0, 15]}
Almost out of time
Max geodes 38
[8, 16, 38]
4864
